In [ ]:
%load_ext autoreload
%autoreload 2
from argparse import Namespace
import requests
import time
import datetime as datetime
from datetime import timedelta
import numpy as np
import json
import os
import glob
from minio import Minio
from minio.error import (InvalidResponseError, S3Error)
import sys

sys.path.insert(0, 'appdata/dev')

from kaapana_federated.utils import get_auth_headers, get_minio_client, get_presigend_url

In [ ]:
def get_init_conf():
    return {
        "query": {
            "bool": {
                "must": [
                    {
                        "match_all": {}
                    },
                    {
                        "match_all": {}
                    },
                    {
                        "match_phrase": {
                            "00120020 ClinicalTrialProtocolID_keyword.keyword": {
                                "query": "segmentations"
                            }
                        }
                    }
                ],
                "filter": [],
                "should": [],
                "must_not": []
            }
        },
        "index": "meta-index",
        "dag": "dev-federated",
        "cohort_limit": 2,
        "form_data": {
            "task": "Task136_RACOON_260122-Federated",
            "model": "3d_lowres",
            "train_network_trainer": "nnUNetTrainerV2",
            "prep_modalities": "CT",
            "node_uid": "node_uid_125497293966312",
            "shuffle_seed": 0,
            "test_percentage": 0,
            "training_description": "nnUnet Segmentation",
            "body_part": "N/A",
            "train_max_epochs": 50,
            "input": "SEG"
        },
        "rest_call": {
        },
        "federated": {
            "federated_operators": ['local-dev'], 
            "skip_operators": ["workflow-cleaner"],
            "host_network": {
                'username': 'kaapana',
                'password': 'admin',
                'protocol': 'https',
                'host': '10.133.28.53',
                'port': '443',
                'ssl_check': False,
                'client_id': 'kaapana',
                'client_secret': '1c4645f0-e654-45a1-a8b6-cf28790104ea'
            }
        }
    }


In [ ]:
def apply_tar_action(dst_filename, src_dir):
    print(f'Tar {src_dir} to {dst_filename}')
    with tarfile.open(dst_filename, "w:gz") as tar:
        tar.add(src_dir, arcname=os.path.basename(src_dir))

def apply_untar_action(src_filename, dst_dir):
    print(f'Untar {src_filename} to {dst_dir}')
    with tarfile.open(src_filename, "r:gz")as tar:
        tar.extractall(dst_dir)

def update_conf(data, site, minioClient, federated_bucket):
    data['conf']['federated']['site'] = site
    minio_urls = {}
    minio_urls['conf'] = {'GET': get_presigend_url(minioClient, 'GET', federated_bucket, os.path.join(site,'conf.tar.gz')), 'PUT': get_presigend_url(minioClient, 'PUT', federated_bucket,  os.path.join(site,'conf.tar.gz'))}
    for federated_operator in data['conf']['federated']['federated_operators']:
        minio_urls[federated_operator] = {'GET': get_presigend_url(minioClient, 'GET', federated_bucket, os.path.join(site, f'{federated_operator}.tar.gz')), 'PUT': get_presigend_url(minioClient, 'PUT', federated_bucket,  os.path.join(site, f'{federated_operator}.tar.gz'))}
    data['conf']['federated']['minio_urls'] = minio_urls
        
minioClient = get_minio_client('kaapanaminio', 'Kaapana2020')

fl_working_dir = '/appdata/dev/federated'
FEDERATED_BUCKET = 'federated'
sites = {
    'dkfz': {
        'username': 'kaapana',
        'password': 'admin',
        'protocol': 'https',
        'host': '10.133.28.53',
        'port': '443',
        'ssl_check': False,
        'client_id': 'kaapana',
        'client_secret': '1c4645f0-e654-45a1-a8b6-cf28790104ea'
    },
    'essen': {
        'username': 'kaapana',
        'password': 'admin',
        'protocol': 'https',
        'host': '10.133.28.53',
        'port': '443',
        'ssl_check': False,
        'client_id': 'kaapana',
        'client_secret': '1c4645f0-e654-45a1-a8b6-cf28790104ea'
    },
}


if not minioClient.bucket_exists(FEDERATED_BUCKET):
    minioClient.make_bucket(FEDERATED_BUCKET)
    
for rn in range(0, 2):
    updated = {site: False for site in sites.keys()}
    fl_working_dir_round = os.path.join(fl_working_dir, str(rn))
    # Starting round!
    now = datetime.datetime.now(tz=datetime.timezone.utc)
    for site, site_info in sites.items():
        if rn == 0:
            conf = get_init_conf()
            conf['federated']['from_previous_dag_run'] =  None
            conf['federated']['rounds'] = [0] 
        else:
            with open(os.path.join(os.path.join(fl_working_dir, str(rn-1)), site, 'conf', 'conf.json'), "r", encoding='utf-8') as jsonData:
                conf = json.load(jsonData)
        meta_data = {
            'conf': conf
        }
        update_conf(meta_data, site, minioClient, FEDERATED_BUCKET)
        print(f'{site_info["protocol"]}://{site_info["host"]}:{site_info["port"]}')
        r = requests.post(f'{site_info["protocol"]}://{site_info["host"]}:{site_info["port"]}/federated-backend/trigger-workflow', json=meta_data,  verify=False,  headers=get_auth_headers(**site_info))
        print(r.text)
        print(site, r.status_code)

    # Waiting for updated files
    for idx in range(20):
        time.sleep(0.5) 
        for site, site_info in sites.items():
            for obj in minioClient.list_objects(FEDERATED_BUCKET, os.path.join(site, 'conf')):
                if now < obj.last_modified:
                    updated[site] = True
        if np.sum(list(updated.values())) == len(sites):
            break
            
    if bool(np.sum(list(updated.values())) == len(sites)) is False:
        print('Update list')
        for k, v in updated.items():
            print(k, v)
        raise ValueError('There are lacking updates, please check what is going on!')

    # Downloading all objects
    for site, site_info in sites.items():
        objects = minioClient.list_objects(FEDERATED_BUCKET, site, recursive=True)
        for obj in objects:
            # https://github.com/minio/minio-py/blob/master/minio/datatypes.py#L103
            if obj.is_dir:
                pass
            else:
#                 file_dir = os.path.join(fl_working_dir_round, os.path.dirname(obj.object_name))
                file_path = os.path.join(fl_working_dir_round, obj.object_name)
                file_dir = os.path.dirname(file_path)
                os.makedirs(file_dir, exist_ok=True)
                minioClient.fget_object(FEDERATED_BUCKET,  obj.object_name, file_path)
#                 dst_dir = os.path.join(file_dir, os.path.basename(obj.object_name).replace('.tar.gz', ''))
                apply_untar_action(file_path, file_dir)
     # Working with downloaded objects

    print('Finished round', rn)

In [ ]:
r = requests.get('https://10.133.28.53/federated-backend/remote/health-check', verify=False, headers={'FederatedAuthorization': '123'})
print(r)
r.text